<a href="https://colab.research.google.com/github/pedrorotta/procesamiento_de_imagenes/blob/main/Detecci%C3%B3n_de_objetos_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#configuración
#Conectar google drive: google.colab import drive /content/drive

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
root = '/content/drive/MyDrive/Curso procesamiento'

In [ ]:
# El comando chdir selecciona un nuevo directorio principal
os.chdir(root)
#Con el comando listdir podemos observar las carpetas o archivos de dicho directorio
#os.listdir()
#list_dir = os.listdir()

In [ ]:
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
from PIL import Image

#**Funciones propias**

In [ ]:
def showrgb(image = None, title = 'Image', size = 4 ):
  #Creada para poder visualizar la imágen rgb. 
  w,h = image.shape[1],image.shape[0]
  aspect_ratio = w/h
  plt.figure(figsize = (size*aspect_ratio,size))
  plt.imshow(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
  plt.title(title)
  plt.show()

#**Detección de objetos I.**

En este capítulo exploraremos los siguientes objetivos:


## Objetivos:

*  Transformada de Fourier
*  Filtros en el dominio de la frecuencia
*  Contornos 
*  Momentos
*  Blob detection